In [1]:
#Mount google drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
%cd /content/gdrive/MyDrive/CV_Final_Project
!ls


/content/gdrive/MyDrive/CV_Final_Project
README.md  s5.zip  Video_to_faceonly.ipynb  Video_to_frame.ipynb


In [5]:
# Import the necessary libraries
import json
import glob
import numpy as np
import cv2
import copy

In [4]:
import zipfile
import os

# Define the compressed file and decompression target path
zip_file_path = "/content/gdrive/MyDrive/CV_Final_Project/s5.zip"
extract_to_path = "/content/gdrive/MyDrive/CV_Final_Project/s5"

# Unzip the file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to_path)

print("Extraction complete. Files are available in:", extract_to_path)

# Verify the decompressed content
!ls -lh "/content/gdrive/MyDrive/CV_Final_Project/s5"


Extraction complete. Files are available in: /content/gdrive/MyDrive/CV_Final_Project/s5
total 4.0K
drwx------ 2 root root 4.0K Nov 18 13:33 s5


This one don't need to run

In [ ]:
#To get the average frame count
#change the path accordingly
video_files =  glob.glob('fake/*.mp4')
#video_files1 =  glob.glob('/content/dfdc_train_part_0/*.mp4')
#video_files += video_files1
frame_count = []
for video_file in video_files:
  cap = cv2.VideoCapture(video_file)
  if(int(cap.get(cv2.CAP_PROP_FRAME_COUNT))<150):
    video_files.remove(video_file)
    continue
  frame_count.append(int(cap.get(cv2.CAP_PROP_FRAME_COUNT)))
print("frames" , frame_count)
print("Total number of videos: " , len(frame_count))
print('Average frame per video:',np.mean(frame_count))

frames [300, 300, 300, 301, 300, 301, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 301, 300, 300, 300, 300, 300, 301, 300, 300, 300, 300, 167, 300, 167, 300, 300, 301, 301, 301, 301, 301, 300, 300, 301, 300, 300]
Total number of videos:  44
Average frame per video: 294.1818181818182


In [6]:
!mkdir -p /content/gdrive/MyDrive/CV_Final_Project/processed_videos

In [8]:
# Import the necessary libraries for video pre-processing
# !pip3 install face_recognition
import torch
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Dataset
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
import face_recognition
from tqdm.autonotebook import tqdm
import gc

In [ ]:
# # to extract frame
# def frame_extract(path):
#   vidObj = cv2.VideoCapture(path)
#   success = 1
#   while success:
#       success, image = vidObj.read()
#       if success:
#           yield image

# # process the frames
# def create_face_videos(path_list,out_dir):
#   already_present_count =  glob.glob(out_dir+'*.mp4')
#   print("No of videos already present " , len(already_present_count))
#   for path in tqdm(path_list):
#     out_path = os.path.join(out_dir,path.split('/')[-1])
#     file_exists = glob.glob(out_path)
#     if(len(file_exists) != 0):
#       print("File Already exists: " , out_path)
#       continue
#     frames = []
#     flag = 0
#     face_all = []
#     frames1 = []
#     out = cv2.VideoWriter(out_path,cv2.VideoWriter_fourcc('M','J','P','G'), 30, (112,112))
#     for idx,frame in enumerate(frame_extract(path)):
#       #if(idx % 3 == 0):
#       if(idx <= 150):
#         frames.append(frame)
#         if(len(frames) == 4):
#           faces = face_recognition.batch_face_locations(frames)
#           for i,face in enumerate(faces):
#             if(len(face) != 0):
#               top,right,bottom,left = face[0]
#             try:
#               out.write(cv2.resize(frames[i][top:bottom,left:right,:],(112,112)))
#             except:
#               pass
#           frames = []
#     try:
#       del top,right,bottom,left
#     except:
#       pass
#     out.release()

In [9]:
path_list = glob.glob('/content/gdrive/MyDrive/CV_Final_Project/s5/*.mp4') + glob.glob('/content/gdrive/MyDrive/CV_Final_Project/s5/*.avi')
frame_counts = []
for path in path_list:
    cap = cv2.VideoCapture(path)
    frame_counts.append(int(cap.get(cv2.CAP_PROP_FRAME_COUNT)))
    cap.release()

print("Max frame count:", max(frame_counts))
print("Min frame count:", min(frame_counts))
print("Average frame count:", sum(frame_counts) / len(frame_counts))

Max frame count: 5444
Min frame count: 5398
Average frame count: 5411.4


In [10]:
def create_face_videos(path_list, out_dir):
    """
    Extract faces from videos and save them as new videos (without frame count limit).

    Args:
        path_list (list): List of paths to input video files.
        out_dir (str): Path to the output directory where processed videos will be saved.
    """
    # Check how many videos already exist in the output directory
    already_present_count = len(glob.glob(os.path.join(out_dir, '*.mp4'))) + len(glob.glob(os.path.join(out_dir, '*.avi')))
    print("Number of videos already present:", already_present_count)

    # Iterate through all video files in the input path list
    for path in tqdm(path_list):
        # Create the output path using the original file name
        out_path = os.path.join(out_dir, os.path.basename(path))
        if os.path.exists(out_path):
            print("File already exists:", out_path)
            continue

        # Open the video file
        vidObj = cv2.VideoCapture(path)
        if not vidObj.isOpened():
            print(f"Error: Unable to open video {path}")
            continue

        # Create a VideoWriter object to save the processed video
        out = cv2.VideoWriter(out_path, cv2.VideoWriter_fourcc('M','J','P','G'), 30, (112, 112))
        frames = []  # To store a batch of frames

        try:
            success, frame = vidObj.read()
            while success:
                frames.append(frame)  # Add the current frame to the batch
                if len(frames) == 4:  # Process every 4 frames as a batch
                    # Perform face detection on the batch of frames
                    faces = face_recognition.batch_face_locations(frames, number_of_times_to_upsample=0, batch_size=2)
                    for i, face_locations in enumerate(faces):
                        for top, right, bottom, left in face_locations:
                            try:
                                # Crop the face from the frame and resize to 112x112
                                cropped_frame = cv2.resize(frames[i][top:bottom, left:right], (112, 112))
                                out.write(cropped_frame)  # Write the cropped face to the output video
                            except Exception as e:
                                print(f"Error cropping frame: {e}")
                    frames.clear()  # Clear the frames list to free up memory
                success, frame = vidObj.read()  # Read the next frame
        except Exception as e:
            print(f"Error processing video {path}: {str(e)}")

        # Release resources after processing the video
        out.release()
        vidObj.release()
        cv2.destroyAllWindows()
        gc.collect()  # Explicitly call garbage collection


In [11]:
# Define the input video file paths (supports both .mp4 and .avi)
path_list = glob.glob('/content/gdrive/MyDrive/CV_Final_Project/s5/*.mp4') + \
            glob.glob('/content/gdrive/MyDrive/CV_Final_Project/s5/*.avi')

# Define the output directory
out_dir = '/content/gdrive/MyDrive/CV_Final_Project/processed_videos'

# Ensure the output directory exists
os.makedirs(out_dir, exist_ok=True)

# Process videos
create_face_videos(path_list, out_dir)

# Check the output directory
!ls -lh /content/gdrive/MyDrive/CV_Final_Project/processed_videos

Number of videos already present: 0


  0%|          | 0/10 [00:00<?, ?it/s]

total 65M
-rw------- 1 root root 7.9K Nov 18 14:19 'gA_2_s5_2019-03-13T09;19;23+01;00_depth_body.avi'
-rw------- 1 root root  23K Nov 18 14:21 'gA_2_s5_2019-03-13T09;19;23+01;00_depth_face.avi'
-rw------- 1 root root 5.6K Nov 18 14:18 'gA_2_s5_2019-03-13T09;19;23+01;00_depth_hands.avi'
-rw------- 1 root root  14M Nov 18 14:06 'gA_2_s5_2019-03-13T09;19;23+01;00_ir_body.mp4'
-rw------- 1 root root  10M Nov 18 14:11 'gA_2_s5_2019-03-13T09;19;23+01;00_ir_face.mp4'
-rw------- 1 root root  258 Nov 18 14:04 'gA_2_s5_2019-03-13T09;19;23+01;00_ir_hands.mp4'
-rw------- 1 root root  11M Nov 18 14:08 'gA_2_s5_2019-03-13T09;19;23+01;00_rgb_body.mp4'
-rw------- 1 root root  13M Nov 18 14:13 'gA_2_s5_2019-03-13T09;19;23+01;00_rgb_face.mp4'
-rw------- 1 root root  258 Nov 18 14:16 'gA_2_s5_2019-03-13T09;19;23+01;00_rgb_hands.mp4'
-rw------- 1 root root  19M Nov 18 14:23 'gA_2_s5_2019-03-13T09;19;23+01;00_rgb_mosaic.avi'
